In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense, Flatten, LeakyReLU, Input, BatchNormalization, Reshape, Conv2D, Conv2DTranspose
from keras.optimizers import Adam, SGD
from keras import Model, Sequential
import os
import cv2

Using TensorFlow backend.


In [16]:
class GANS():
    def __init__(self):
        keras.backend.clear_session()
        self.img_rows = 120
        self.img_cols = 120
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        optimizer = Adam(0.0002, 0.8)
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
        
        self.generator = self.build_generator()
        self.generator.compile(loss = 'binary_crossentropy', optimizer = optimizer)
        
        z = Input(shape = (5, 5, 3))
        img = self.generator(z)
        
        
        self.discriminator.trainable = False
        
        #combined model will disallow trained values from the discriminator
        valid = self.discriminator(img)
        
        
        self.combined = Model(z, valid)
        self.combined.compile(loss= 'binary_crossentropy', optimizer = optimizer)
        
    def build_discriminator(self):
        img_shape = (self.img_rows, self.img_cols, self.channels)
        
        model = Sequential()
        
        model.add(Conv2D(64, kernel_size = [5, 5], strides = 2, input_shape = img_shape))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Conv2D(128, kernel_size = [5, 5], strides = 2))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Conv2D(256, kernel_size = [5, 5], strides = 2))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Flatten())
        model.add(Dense(1, activation = 'sigmoid'))
        model.summary()
        
        img = Input(shape = img_shape)
        validity = model(img)
        return Model(img, validity)
    
    def build_generator(self):
        
        noise_shape = (5, 5, 3)
        model = Sequential()
        
        model.add(Conv2DTranspose(filters = 128, kernel_size = (5,5), strides = 2, input_shape = noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum = 0.8))
        model.add(Conv2DTranspose(filters = 256, kernel_size = (5,5), strides = 2))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(BatchNormalization(momentum = 0.8))
        model.add(Conv2DTranspose(filters = 512, kernel_size = (3, 3),  strides = 2))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(BatchNormalization(momentum = 0.8))
        model.add(Conv2DTranspose(filters = 1024, kernel_size = (3, 3),  strides = 2 ))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(BatchNormalization(momentum = 0.8))
        model.add(Conv2DTranspose(filters = 3, kernel_size = (2, 2), strides = 1, activation = 'tanh'))

        model.summary()

        noise = Input(shape = noise_shape)
        img = model(noise)
        return Model(noise, img)
    
    def train(self, epochs, batch_size = 16, save_interval=50):
        #load the data
        X_train = self.process_images()
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        #rescale img data to -1 to 1
        half_batch = int(batch_size/2)
        
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]
            imgs = imgs.reshape(8, 120, 120, 3)
            noise = np.random.normal(0, 1, (half_batch, 5, 5, 3))
            
            gen_imgs = self.generator.predict(noise)
            gen_imgs = gen_imgs.reshape(8, 120, 120, 3)
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            #train generator
            
            noise = np.random.normal(0, 1 , (batch_size, 5, 5, 3))
            
            valid_y = np.array([1] * batch_size)
            
            g_loss = self.combined.train_on_batch(noise, valid_y)
            
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))            
            
            if epoch % save_interval == 0:
                self.save_imgs(epoch)
    def save_imgs(self, epoch):
        noise = np.random.normal(0, 1, (1, 5, 5, 3))
        gen_imgs = self.generator.predict(noise)
        #print(gen_imgs)
        gen_imgs = (gen_imgs + 0.5) * 255
        
        cv2.imwrite("imgs\\img_" + str(epoch) + ".jpg", gen_imgs[0])
        
        plt.close()
    
    def process_images(self):
        imgs = []
        path = "flower_images\\"
        for i in os.listdir(path):
            im = cv2.imread(path + i)
            im = cv2.resize(im, (120, 120))
            imgs.append(im)
        imgs = np.asarray(imgs) 
        return imgs
    
if __name__ == '__main__':
    gan = GANS()
    gan.train(epochs = 1000000, batch_size = 16, save_interval =100)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 58, 58, 64)        4864      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 58, 58, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 128)       204928    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 27, 27, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 256)       819456    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 12, 12, 256)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 36864)             0         
__________

86 [D loss: 0.264209, acc.: 100.00%] [G loss: 2.631515]
87 [D loss: 0.284932, acc.: 87.50%] [G loss: 2.926334]
88 [D loss: 0.115990, acc.: 100.00%] [G loss: 3.173521]
89 [D loss: 0.098652, acc.: 100.00%] [G loss: 3.774555]
90 [D loss: 0.146097, acc.: 100.00%] [G loss: 3.975876]
91 [D loss: 0.197541, acc.: 87.50%] [G loss: 3.144850]
92 [D loss: 0.190715, acc.: 100.00%] [G loss: 3.521389]
93 [D loss: 0.135436, acc.: 100.00%] [G loss: 3.865468]
94 [D loss: 0.047021, acc.: 100.00%] [G loss: 4.383481]
95 [D loss: 0.084685, acc.: 100.00%] [G loss: 4.352734]
96 [D loss: 0.044537, acc.: 100.00%] [G loss: 4.285018]
97 [D loss: 0.044861, acc.: 100.00%] [G loss: 5.158152]
98 [D loss: 0.079158, acc.: 100.00%] [G loss: 4.878681]
99 [D loss: 0.313762, acc.: 87.50%] [G loss: 5.314603]
100 [D loss: 0.947885, acc.: 68.75%] [G loss: 8.277867]
101 [D loss: 0.183878, acc.: 87.50%] [G loss: 11.952009]
102 [D loss: 1.142241, acc.: 68.75%] [G loss: 3.040592]
103 [D loss: 0.659594, acc.: 75.00%] [G loss: 1.87

231 [D loss: 0.103468, acc.: 100.00%] [G loss: 3.631385]
232 [D loss: 0.023496, acc.: 100.00%] [G loss: 5.620250]
233 [D loss: 0.052287, acc.: 100.00%] [G loss: 6.646773]
234 [D loss: 0.086240, acc.: 93.75%] [G loss: 5.018990]
235 [D loss: 0.219485, acc.: 87.50%] [G loss: 4.094885]
236 [D loss: 0.234236, acc.: 87.50%] [G loss: 7.283366]
237 [D loss: 0.052810, acc.: 100.00%] [G loss: 10.978909]
238 [D loss: 0.322116, acc.: 81.25%] [G loss: 8.303231]
239 [D loss: 0.022187, acc.: 100.00%] [G loss: 5.366809]
240 [D loss: 0.175724, acc.: 93.75%] [G loss: 6.211028]
241 [D loss: 0.006406, acc.: 100.00%] [G loss: 5.897692]
242 [D loss: 0.030602, acc.: 100.00%] [G loss: 5.627195]
243 [D loss: 0.071083, acc.: 100.00%] [G loss: 5.129201]
244 [D loss: 0.070647, acc.: 100.00%] [G loss: 4.737850]
245 [D loss: 0.038640, acc.: 100.00%] [G loss: 5.683996]
246 [D loss: 0.018575, acc.: 100.00%] [G loss: 5.682915]
247 [D loss: 0.099104, acc.: 93.75%] [G loss: 6.079000]
248 [D loss: 0.093070, acc.: 100.00%

376 [D loss: 0.018888, acc.: 100.00%] [G loss: 5.436790]
377 [D loss: 0.013048, acc.: 100.00%] [G loss: 5.562320]
378 [D loss: 0.030858, acc.: 100.00%] [G loss: 5.316233]
379 [D loss: 0.012785, acc.: 100.00%] [G loss: 5.190600]
380 [D loss: 0.023458, acc.: 100.00%] [G loss: 4.440247]
381 [D loss: 0.214050, acc.: 87.50%] [G loss: 5.538223]
382 [D loss: 0.004072, acc.: 100.00%] [G loss: 9.733889]
383 [D loss: 0.211938, acc.: 81.25%] [G loss: 7.010839]
384 [D loss: 0.011450, acc.: 100.00%] [G loss: 5.594161]
385 [D loss: 0.156118, acc.: 87.50%] [G loss: 3.093510]
386 [D loss: 0.068025, acc.: 100.00%] [G loss: 4.182141]
387 [D loss: 0.011992, acc.: 100.00%] [G loss: 4.820352]
388 [D loss: 0.094172, acc.: 93.75%] [G loss: 4.504214]
389 [D loss: 0.014180, acc.: 100.00%] [G loss: 4.364476]
390 [D loss: 0.028204, acc.: 100.00%] [G loss: 4.953109]
391 [D loss: 0.006868, acc.: 100.00%] [G loss: 5.850537]
392 [D loss: 0.007465, acc.: 100.00%] [G loss: 6.016589]
393 [D loss: 0.037962, acc.: 100.00

521 [D loss: 0.105350, acc.: 93.75%] [G loss: 5.910686]
522 [D loss: 0.271365, acc.: 87.50%] [G loss: 3.533575]
523 [D loss: 0.049331, acc.: 100.00%] [G loss: 2.848933]
524 [D loss: 0.066838, acc.: 100.00%] [G loss: 3.453243]
525 [D loss: 0.008277, acc.: 100.00%] [G loss: 4.276259]
526 [D loss: 0.094115, acc.: 93.75%] [G loss: 4.172694]
527 [D loss: 0.030110, acc.: 100.00%] [G loss: 3.816064]
528 [D loss: 0.052764, acc.: 100.00%] [G loss: 5.109405]
529 [D loss: 0.006010, acc.: 100.00%] [G loss: 5.903540]
530 [D loss: 0.225587, acc.: 93.75%] [G loss: 5.121670]
531 [D loss: 0.139730, acc.: 93.75%] [G loss: 4.300133]
532 [D loss: 0.009255, acc.: 100.00%] [G loss: 5.032364]
533 [D loss: 0.008743, acc.: 100.00%] [G loss: 5.981936]
534 [D loss: 0.005895, acc.: 100.00%] [G loss: 5.433773]
535 [D loss: 0.170791, acc.: 93.75%] [G loss: 4.490944]
536 [D loss: 0.035582, acc.: 100.00%] [G loss: 5.998094]
537 [D loss: 0.073531, acc.: 100.00%] [G loss: 5.965854]
538 [D loss: 0.039102, acc.: 100.00%]

...

4667 [D loss: 0.157665, acc.: 93.75%] [G loss: 8.262231]
4668 [D loss: 0.013098, acc.: 100.00%] [G loss: 8.174436]
4669 [D loss: 0.001661, acc.: 100.00%] [G loss: 7.560974]
4670 [D loss: 0.157146, acc.: 93.75%] [G loss: 8.195002]
4671 [D loss: 0.009299, acc.: 100.00%] [G loss: 11.774468]
4672 [D loss: 0.002459, acc.: 100.00%] [G loss: 12.095583]
4673 [D loss: 0.044700, acc.: 100.00%] [G loss: 10.351102]
4674 [D loss: 0.018419, acc.: 100.00%] [G loss: 8.965024]
4675 [D loss: 0.002606, acc.: 100.00%] [G loss: 6.702504]
4676 [D loss: 0.052255, acc.: 100.00%] [G loss: 7.271139]
4677 [D loss: 0.143697, acc.: 93.75%] [G loss: 9.027447]
4678 [D loss: 0.035365, acc.: 100.00%] [G loss: 10.425377]
4679 [D loss: 0.023600, acc.: 100.00%] [G loss: 11.071317]
4680 [D loss: 0.017355, acc.: 100.00%] [G loss: 8.176144]
4681 [D loss: 0.007772, acc.: 100.00%] [G loss: 6.618953]
4682 [D loss: 0.035895, acc.: 100.00%] [G loss: 7.728843]
4683 [D loss: 0.017258, acc.: 100.00%] [G loss: 5.753740]
4684 [D loss

4809 [D loss: 0.013937, acc.: 100.00%] [G loss: 10.348060]
4810 [D loss: 0.022520, acc.: 100.00%] [G loss: 8.487272]
4811 [D loss: 0.015262, acc.: 100.00%] [G loss: 9.259806]
4812 [D loss: 0.098025, acc.: 93.75%] [G loss: 11.039459]
4813 [D loss: 0.010297, acc.: 100.00%] [G loss: 11.700090]
4814 [D loss: 0.020165, acc.: 100.00%] [G loss: 12.757659]
4815 [D loss: 0.009256, acc.: 100.00%] [G loss: 11.048972]
4816 [D loss: 0.010363, acc.: 100.00%] [G loss: 9.936310]
4817 [D loss: 0.018239, acc.: 100.00%] [G loss: 9.375900]
4818 [D loss: 0.002796, acc.: 100.00%] [G loss: 8.827736]
4819 [D loss: 0.061859, acc.: 93.75%] [G loss: 8.391219]
4820 [D loss: 0.002905, acc.: 100.00%] [G loss: 10.106699]
4821 [D loss: 0.001586, acc.: 100.00%] [G loss: 9.269849]
4822 [D loss: 0.037521, acc.: 100.00%] [G loss: 8.656329]
4823 [D loss: 0.003091, acc.: 100.00%] [G loss: 8.669857]
4824 [D loss: 0.008487, acc.: 100.00%] [G loss: 7.940146]
4825 [D loss: 0.014691, acc.: 100.00%] [G loss: 8.200871]
4826 [D lo

4950 [D loss: 0.070616, acc.: 93.75%] [G loss: 7.550548]
4951 [D loss: 0.043087, acc.: 100.00%] [G loss: 9.429914]
4952 [D loss: 0.024721, acc.: 100.00%] [G loss: 9.800175]
4953 [D loss: 0.009836, acc.: 100.00%] [G loss: 10.956890]
4954 [D loss: 0.387118, acc.: 81.25%] [G loss: 4.395069]
4955 [D loss: 0.708417, acc.: 68.75%] [G loss: 9.018676]
4956 [D loss: 0.001045, acc.: 100.00%] [G loss: 14.737044]
4957 [D loss: 0.133271, acc.: 87.50%] [G loss: 15.363968]
4958 [D loss: 0.274719, acc.: 81.25%] [G loss: 13.026635]
4959 [D loss: 0.000219, acc.: 100.00%] [G loss: 11.300836]
4960 [D loss: 0.004721, acc.: 100.00%] [G loss: 7.636213]
4961 [D loss: 0.182015, acc.: 87.50%] [G loss: 8.133007]
4962 [D loss: 0.138841, acc.: 93.75%] [G loss: 10.081497]
4963 [D loss: 0.022675, acc.: 100.00%] [G loss: 12.500538]
4964 [D loss: 0.432633, acc.: 93.75%] [G loss: 10.227892]
4965 [D loss: 0.000093, acc.: 100.00%] [G loss: 10.913452]
4966 [D loss: 0.347564, acc.: 93.75%] [G loss: 7.337955]
4967 [D loss: 

5091 [D loss: 0.018907, acc.: 100.00%] [G loss: 9.435970]
5092 [D loss: 0.001596, acc.: 100.00%] [G loss: 9.769369]
5093 [D loss: 0.012504, acc.: 100.00%] [G loss: 9.630892]
5094 [D loss: 0.007943, acc.: 100.00%] [G loss: 9.882671]
5095 [D loss: 0.034544, acc.: 100.00%] [G loss: 9.466042]
5096 [D loss: 0.001082, acc.: 100.00%] [G loss: 11.724043]
5097 [D loss: 0.000293, acc.: 100.00%] [G loss: 11.443282]
5098 [D loss: 0.062639, acc.: 93.75%] [G loss: 10.179629]
5099 [D loss: 0.009108, acc.: 100.00%] [G loss: 8.374790]
5100 [D loss: 0.002644, acc.: 100.00%] [G loss: 6.425106]
5101 [D loss: 0.215592, acc.: 93.75%] [G loss: 6.825694]
5102 [D loss: 0.002859, acc.: 100.00%] [G loss: 9.742846]
5103 [D loss: 0.019245, acc.: 100.00%] [G loss: 10.567359]
5104 [D loss: 0.240185, acc.: 87.50%] [G loss: 7.596775]
5105 [D loss: 0.033332, acc.: 100.00%] [G loss: 5.620781]
5106 [D loss: 0.009682, acc.: 100.00%] [G loss: 5.303127]
5107 [D loss: 0.150996, acc.: 93.75%] [G loss: 5.876643]
5108 [D loss: 

ResourceExhaustedError: OOM when allocating tensor with shape[16,119,119,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training_1/Adam/gradients/zeros_1-0-1-TransposeNCHWToNHWC-LayoutOptimizer}} = Transpose[T=DT_FLOAT, Tperm=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_1/Adam/gradients/zeros_1, PermConstNCHWToNHWC-LayoutOptimizer)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss_2/mul/_419}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_2237_loss_2/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocati